# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

#### Run shared detector on bash
```bash
python3 /home/kiro-ros/Projects/rnb-planning/src/scripts/demo_202107/demo_utils/shared_detector.py
```

#### Check and request ip setting from mobile udp client (robot-side)

#### 0.1.1 Set parameters and create planning scene

In [1]:
import numpy as np

CONNECT_INDY = True
CONNECT_MOBILE = True

CONNECT_TASK_PLANNER = False
VISUALIZE = False
VERBOSE = False
PLANNING_MULTIPROC = True
N_AGENTS = 10
NODE_TRIAL_MAX = N_AGENTS * 2
MAX_SOL_NUM = 5
BASE_COST_CUT = 110

TIMEOUT_MOTION = 0.5
TIMEOUT_FULL = 5

ROS_MASTER_ON_MOBILE = False
# Tool dimensions
TOOL_DIM = [0.24, 0.35]
TOOL_CENTER_OFF = [0, -0.035, 0]
TOOL_THICKNESS = 0.11 + 0.05

INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 0.16

COL_COLOR = (1,1,1,0.2)

USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()

from pkg.detector.multiICP.multiICP import *
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *
from demo_utils.area_select import DATASET_DIR, SweepDirections
from demo_utils.demo_config import *
from demo_utils.detection_util import *

from pkg.utils.shared_function import *
clear_channels_on("kiromobilemap")

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7kiro, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})
ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

s_builder = SceneBuilder(None)
SceneBuilder.autostart_roscore = not ROS_MASTER_ON_MOBILE
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)
gscene.set_workspace_boundary(
    -1, 2.5, -0.5, 3, -CLEARANCE, 1.6, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=CAM_LINK, center=(-0.0785, 0, 0.073))

add_brush(gscene, face_name=TOOL_NAME, tool_link=TIP_LINK,
          thickness=TOOL_THICKNESS, tool_dim=TOOL_DIM,
          col_color=COL_COLOR, center_off=TOOL_CENTER_OFF)

HOME_POSE = crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, 
                      incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

mplan.reset_PRQdict(enable_PRQ=0.5, radii=5e-2)
for tip_dir, SWEEP_AXIS in [
    (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)+"-PRQ.pkl"
    PRQ_PATH = os.path.join(SWEEP_DATA_DIR, filename)
    try:
        Pos_Rotvec_Qlist_dict = load_pickle(PRQ_PATH)
        mplan.register_PRQ(ROBOT_NAME, Pos_Rotvec_Qlist_dict, decimal=2)
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=NODE_TRIAL_MAX)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, \
    Gripper2Tool, PlacePlane, SweepFramer

brush_face = pscene.create_binder(
    bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
    point=(0,0, -gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
    rpy=(0,0,0))

gscene.create_safe(
    gtype=GEOTYPE.BOX, name="floor_box", link_name="base_link",
    dims=(15,15,0.4), center=(0,0,0), rpy=(0,0,0), 
    color=(1, 1, 1, 0.1), display=True, collision=False, fixed=True)

gscene.add_highlight_axis("hl", "base_coord", T=np.identity(4), dims=(0.5,0.1,0.1))

kmb = crob.robot_dict["kmb0"]
indy = crob.robot_dict["indy1"]
mobile_box = gscene.NAME_DICT['kmb0_platform_Box_2']
crob.simulator.set_gscene(gscene)
# indy.ktool.led_on()
# indy.ktool.led_off()
# time.sleep(2)
# indy.ktool.send_degree(-45)

if CONNECT_MOBILE:
    assert np.sum(np.abs(get_xyzw_cur()))>1e-4, "UDP Server not connected"
    
if CONNECT_TASK_PLANNER:
    from demo_proto.DisinfectionOperationServicer import serve_on_thread
    servicer = serve_on_thread()

connection command:
kmb0: True
indy1: True
[MOBILE ROBOT] bind: ('192.168.0.10', 50306)
[MOBILE ROBOT] Start UDP THREAD
goal reach: 0 -> 1 (1639643393.79)
goal reach signal edge up
==== Kiro Tool connected to /dev/ttyUSB1 (115200) ====
[KTOOL] initialize
[KTOOL] enable
[KTOOL] send (<MOTOR_CMD.ENABLE: 1>, <OP_CMD.NONE: 0>)
01:00:00:00:
[KTOOL] recv [161, 0, 0, 0]
[KTOOL] op_init
[KTOOL] send (<MOTOR_CMD.OP_INIT: 3>, <OP_CMD.NONE: 0>)
03:00:00:00:
[KTOOL] recv [163, 255, 0, 0]
[KTOOL] led_off
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.LED_OFF: 5>)
02:05:00:00:
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Loaded: indy7kiro-frontX-PRQ.pkl
Loaded: indy7kiro-upZ-PRQ.pkl
Loaded: indy7kiro-downZ-PRQ.pkl


#### 0.1.2 Load environment map

In [3]:
from demo_utils.ros_map_utils import KiroMobileMap
kmm = KiroMobileMap(MOBILE_IP, IP_CUR, CONNECT_MOBILE)
            
VALID_BOX_SCALE = 0.8
VALID_SCORE_CUT = 50
kmb.coster = (lambda Q: 
                  np.max(
                      kmm.get_box_costs(mobile_box, Q, kmm.T_bi, kmm.cost_im, kmm.resolution, 
                                        scale=VALID_BOX_SCALE)))
kmb.cost_cut = VALID_SCORE_CUT
kmb.gscene = gscene

kmm.init_node(timeout=10)

In [4]:
pole_pt_list, pole_res = kmm.update_map(gscene, crob, MOBILE_BASE, timeout=100)
if not CONNECT_MOBILE:
    crob.joint_move_make_sure(kmm.Q_map)
    
if CONNECT_INDY:
    indy.move_joint_s_curve(crob.home_pose[6:], N_div=500)
else:
    indy.joint_move_make_sure(crob.home_pose[6:])

Qcur = crob.get_real_robot_pose()

[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.
Connect: Server IP (192.168.0.3)
Move Tool: -0.000444130736915 (1639643410.17)
[KTOOL] send degree: 0.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
02:06:00:00:
Move Tool: -0.00177650541414 (1639643410.21)
[KTOOL] send degree: -0.000444130736915
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
02:06:00:00:
Move Tool: -0.00710574112575 (1639643410.25)
[KTOOL] send degree: -0.00177650541414
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
02:06:00:00:
Move Tool: -0.0159868655867 (1639643410.29)
[KTOOL] send degree: -0.00710574112575
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
02:06:00:00:
Move Tool: -0.0284184763646 (1639643410.33)
[KTOOL] send degree: -0.0159868655867
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
02:06:00:00:
Move Tool: -0.0443986103639 (1639643410.37)
[KTOOL] send degree: -0.0284184763646
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.D

Move Tool: -5.436856906 (1639643412.43)
[KTOOL] send degree: -5.25390734679
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 204)
02:06:255:204:
Move Tool: -5.62250093331 (1639643412.46)
[KTOOL] send degree: -5.436856906
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 202)
02:06:255:202:
Move Tool: -5.81081011338 (1639643412.5)
[KTOOL] send degree: -5.62250093331
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 200)
02:06:255:200:
Move Tool: -6.00175470999 (1639643412.54)
[KTOOL] send degree: -5.81081011338
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 198)
02:06:255:198:
Move Tool: -6.19530457079 (1639643412.58)
[KTOOL] send degree: -6.00175470999
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 196)
02:06:255:196:
Move Tool: -6.39142913201 (1639643412.62)
[KTOOL] send degree: -6.19530457079
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 195)
02:06:255:195:
Move Tool: -6.5900974233 (1639643412.66)
[KTOOL] send degree: -6.3914

Move Tool: -19.1199242448 (1639643414.68)
[KTOOL] send degree: -18.8406637831
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 68)
02:06:255:68:
Move Tool: -19.3997184596 (1639643414.72)
[KTOOL] send degree: -19.1199242448
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 65)
02:06:255:65:
Move Tool: -19.6800022448 (1639643414.76)
[KTOOL] send degree: -19.3997184596
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 63)
02:06:255:63:
Move Tool: -19.9607313403 (1639643414.8)
[KTOOL] send degree: -19.6800022448
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 60)
02:06:255:60:
Move Tool: -20.2418614158 (1639643414.84)
[KTOOL] send degree: -19.9607313403
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 57)
02:06:255:57:
Move Tool: -20.5233480776 (1639643414.88)
[KTOOL] send degree: -20.2418614158
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 54)
02:06:255:54:
Move Tool: -20.8051468756 (1639643414.91)
[KTOOL] send degree: -20.5233480776

Move Tool: -34.6752281729 (1639643416.99)
[KTOOL] send degree: -34.4365016469
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 168)
02:06:254:168:
Move Tool: -34.9120320891 (1639643417.03)
[KTOOL] send degree: -34.6752281729
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 166)
02:06:254:166:
Move Tool: -35.1468760017 (1639643417.07)
[KTOOL] send degree: -34.9120320891
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 163)
02:06:254:163:
Move Tool: -35.3797228259 (1639643417.11)
[KTOOL] send degree: -35.1468760017
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 161)
02:06:254:161:
Move Tool: -35.6105357925 (1639643417.14)
[KTOOL] send degree: -35.3797228259
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 159)
02:06:254:159:
Move Tool: -35.8392784536 (1639643417.18)
[KTOOL] send degree: -35.6105357925
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 156)
02:06:254:156:
Move Tool: -36.065914688 (1639643417.22)
[KTOOL] send degree: -3

Move Tool: -44.1837827615 (1639643419.31)
[KTOOL] send degree: -44.1066079277
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 71)
02:06:254:71:
Move Tool: -44.2575334747 (1639643419.35)
[KTOOL] send degree: -44.1837827615
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 71)
02:06:254:71:
Move Tool: -44.327848421 (1639643419.39)
[KTOOL] send degree: -44.2575334747
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 70)
02:06:254:70:
Move Tool: -44.3947164971 (1639643419.43)
[KTOOL] send degree: -44.327848421
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 69)
02:06:254:69:
Move Tool: -44.4581271436 (1639643419.47)
[KTOOL] send degree: -44.3947164971
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 69)
02:06:254:69:
Move Tool: -44.5180703473 (1639643419.51)
[KTOOL] send degree: -44.4581271436
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 68)
02:06:254:68:
Move Tool: -44.5745366425 (1639643419.55)
[KTOOL] send degree: -44.5180703473


In [5]:
# pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res, height=WS_HEIGHT)
# gcheck.ignore_always = pole_list

# gscene.update_markers_all() 

## 1. Prepare scene

In [6]:
HEIGHT_OFFSET = 0.02
TABLE_DIMS = (0.60,1.2, 0.74+HEIGHT_OFFSET)
TABLE_LOC = (0,1.85,TABLE_DIMS[2]/2)
TABLE_ANG = 0
TABLE_MARGIN_I = 0.03
TABLE_MARGIN_O = 0.03
CLEARANCE = 0.002
TABLE_COLOR = (0.8,0.8,0.8,1)
SURF_COLOR = (0,1,0,0.4)
COL_COLOR = (1,1,1,0.2)

gscene.create_safe(GEOTYPE.BOX, "table", link_name="base_link",
                   dims=TABLE_DIMS, center=TABLE_LOC, rpy=Rot2rpy(Rot_axis(3, TABLE_ANG)),
                   color=TABLE_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "table_col", link_name="base_link",
                   dims=np.add(TABLE_DIMS, 
                               (TABLE_MARGIN_O*2, TABLE_MARGIN_O*2, 0)), 
                   center=(0,0,0), rpy=(0,0,0), color=COL_COLOR, 
                   collision=True, parent="table"
                  )
table_surf = gscene.create_safe(GEOTYPE.BOX, "table_surf", link_name="base_link",
                   dims=np.subtract(TABLE_DIMS[:2]+(CLEARANCE,), 
                                    (TABLE_MARGIN_I*2, TABLE_MARGIN_I*2, 0)), 
                   center=(0,0,(TABLE_DIMS[2]+CLEARANCE)/2), 
                   rpy=(0,0,0), color=SURF_COLOR, parent="table",
                   collision=False
                  )

In [7]:
SOFA_DIMS = (0.49,1.0, 0.43)
SOFA_LOC = (0.15,0.7,SOFA_DIMS[2]/2+HEIGHT_OFFSET)
SOFA_SIDE_THIC = 0.2
SOFA_SIDE_HEIT = 0.54
SOFA_ANG = 0

SOFA_MARGIN_O = 0.03
SOFA_MARGIN_I = 0.05
SOFA_UV_DIST = 0.15

SOFA_COLOR = (0.4,0.4,0.4,1)

sofa = gscene.create_safe(GEOTYPE.BOX, "sofa", link_name="base_link",
                   dims=SOFA_DIMS, center=SOFA_LOC, 
                   rpy=Rot2rpy(Rot_axis(3, SOFA_ANG)),
                   color=SOFA_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_col", link_name="base_link",
                   dims=np.add(SOFA_DIMS, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa"
                  )
# sofa_left = gscene.create_safe(GEOTYPE.BOX, "sofa_left", link_name="base_link",
#                    dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    center=(0,(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
#                    rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
#                   )
# gscene.create_safe(GEOTYPE.BOX, "sofa_left_col", link_name="base_link",
#                    dims=np.add(sofa_left.dims, SOFA_MARGIN_O*2), 
#                    center=(0,0,0), rpy=(0,0,0),
#                    color=COL_COLOR, collision=True, parent="sofa_left"
#                   )
# sofa_right = gscene.create_safe(GEOTYPE.BOX, "sofa_right", link_name="base_link",
#                    dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    center=(0,-(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
#                    rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
#                   )
# gscene.create_safe(GEOTYPE.BOX, "sofa_right_col", link_name="base_link",
#                    dims=np.add(sofa_right.dims, SOFA_MARGIN_O*2), 
#                    center=(0,0,0), rpy=(0,0,0),
#                    color=COL_COLOR, collision=True, parent="sofa_right"
#                   )
sofa_back = gscene.create_safe(GEOTYPE.BOX, "sofa_back", link_name="base_link",
#                    dims=(SOFA_SIDE_THIC, SOFA_DIMS[1]+SOFA_SIDE_THIC*2, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   dims=(SOFA_SIDE_THIC, SOFA_DIMS[1], SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(-(SOFA_DIMS[0]+SOFA_SIDE_THIC)/2, 0, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_back_col", link_name="base_link",
                   dims=np.add(sofa_back.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_back"
                  )

sofa_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_surf", link_name="base_link",
                               dims=tuple(np.subtract(SOFA_DIMS[:2], SOFA_MARGIN_I*2))+(CLEARANCE,), 
                               center=(0,0,SOFA_DIMS[2]/2+SOFA_UV_DIST), collision=False, 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_back_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_back_surf", link_name="base_link",
                               dims=sofa_back.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(SOFA_MARGIN_I*2.5,0,sofa_back.dims[2]/2+SOFA_MARGIN_O+CLEARANCE), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_back"
                              )

# sofa_left_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_left_surf", link_name="base_link",
#                                dims=sofa_left.dims[:2]+(CLEARANCE,), collision=False, 
#                                center=(0,0,sofa_left.dims[2]/2+SOFA_UV_DIST), 
#                                rpy=(0,0,0), color=SURF_COLOR, parent="sofa_left"
#                               )

# sofa_right_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_right_surf", link_name="base_link",
#                                dims=sofa_right.dims[:2]+(CLEARANCE,), collision=False, 
#                                center=(0,0,sofa_right.dims[2]/2+SOFA_UV_DIST), 
#                                rpy=(0,0,0), color=SURF_COLOR, parent="sofa_right"
#                               )

sofa_front = gscene.create_safe(GEOTYPE.BOX, "sofa_front", link_name="base_link",
                               dims=(sofa.dims[2], sofa.dims[1])+(CLEARANCE,), 
                               center=(sofa.dims[0]/2+SOFA_UV_DIST,0,SOFA_MARGIN_I), collision=False, 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa"
                              )

# sofa_left_front = gscene.create_safe(GEOTYPE.BOX, "sofa_left_front", link_name="base_link",
#                                dims=(sofa_left.dims[2], sofa_left.dims[1])+(CLEARANCE,), 
#                                center=(sofa_left.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
#                                rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_left"
#                               )

# sofa_right_front = gscene.create_safe(GEOTYPE.BOX, "sofa_right_front", link_name="base_link",
#                                dims=(sofa_right.dims[2], sofa_right.dims[1])+(CLEARANCE,), 
#                                center=(sofa_right.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
#                                rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_right"
#                               )

sofa_back_front = gscene.create_safe(GEOTYPE.BOX, "sofa_back_front", link_name="base_link",
#                                dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_SIDE_THIC*2-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               center=(sofa_back.dims[0]/2+SOFA_UV_DIST,0,SOFA_DIMS[2]/2+SOFA_MARGIN_I*1.5), 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, collision=False, parent="sofa_back"
                              )

In [8]:
pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["room_ws"], 
                    pole_pt_list, Qcur, inside=False, margin=0.1)
pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["table_col"], 
                    pole_pt_list, Qcur, margin=0.1)
pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_col"], 
                    pole_pt_list, Qcur, margin=0.1)
# pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_back_col"], 
#                     pole_pt_list, Qcur, margin=0.1)
# pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_right_col"], 
#                     pole_pt_list, Qcur, margin=0.1)

pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res, height=WS_HEIGHT)
gcheck.ignore_always = pole_list

gscene.update_markers_all() 

### Set UI

In [9]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


### 1.0 Wait task start queue

In [10]:
if CONNECT_TASK_PLANNER:
    while servicer.object_info_running.object_id < 0:
        time.sleep(1)

   Use a production WSGI server instead.
 * Debug mode: off



## 2. Prepare cleaning

In [11]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *
from pkg.utils.code_scraps import get_look_motion

In [12]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
Qcur[6:] = crob.home_pose[6:]
crob.joint_move_make_sure(Qcur)
Qcur = crob.get_real_robot_pose()
if CONNECT_INDY:
    indy.ktool.led_off()

Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -45.0 (1639643424.57)
[KTOOL] send degree: -45.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
02:06:254:62:
[KTOOL] led_off
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.LED_OFF: 5>)
02:05:00:00:


In [13]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
if not CONNECT_INDY:
    Qcur[6:] = crob.home_pose[6:]
crob.joint_move_make_sure(Qcur)
HOME_POSE_SWEEP = np.copy(Qcur)
# HOME_POSE_SWEEP[6:] = 0
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, 
                           gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    

VEL_LIMS = 0.4
ACC_LIMS = 0.4
SWP_VEL_LIMS = 0.2
SWP_ACC_LIMS = 0.2
RADI_DEG = 1

if CONNECT_INDY:
    indy.QVEL_LEVEL = 3
#     indy.collision_policy = indy_trajectory_client_nosdk.POLICY_NO_COLLISION_DETECTION
    indy.reset()

gtimer = GlobalTimer()
    
mode_switcher=ModeSwitcherLED(pscene, robot_name=ROBOT_NAME, brush_face=brush_face.geometry)

def no_offset(gxter, crob, mplan, robot_name, Qref):
    return Qref, Qref

def cleaning_fun(surface, tip_dir, sweep_dir, tool_dir, Qcur,
                 swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS):
    if sweep_dir.lower() == "z":
        EE_HEIGHT = None
    else:
        T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
        T_brush_e = SE3_inv(T_e_brush)
        EE_HEIGHT = round(surface.get_tf(HOME_DICT)[2,3] + surface.dims[2]/2, 5) \
                        + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
    gtimer.reset(scale=1, timeunit='s', stack=True)
    gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                           vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                           swp_vel_lims=swp_vel_lims, swp_acc_lims=swp_acc_lims
                          )

    gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                          timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                          verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

    gxter.get_division_dict(surface, tip_dir, sweep_dir, EE_HEIGHT)
    gxter.init_base_divs(Qcur)
    # gxter.mark_tested(None, None, covereds_all, [])
    snode_schedule_list, Qcur, covereds = gxter.greedy_execute(
        Qcur, tool_dir, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

    gxter.test_clear()
    print(gtimer)
    gscene.clear_virtuals()
    return snode_schedule_list, Qcur

Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -45.0 (1639643426.17)
[KTOOL] send degree: -45.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
02:06:254:62:
Connect: Server IP (192.168.0.3)


## 3. Table cleaning

In [14]:
snode_schedule_list_table_all = []

In [15]:
snode_schedule_list_table, Qcur = cleaning_fun(surface=table_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
snode_schedule_list_table_all.append(snode_schedule_list_table)

('Height Reference: ', 0.48399999845027925)
Send to mobile: [ 1.114  1.545 -1.    -0.011]
             -> [ 1.027  1.552  1.     0.   ]
Distance=0.0899268119244 ([-0.087  0.007  0.022])
goal reach: 1 -> 0 (1639643429.69)
goal reach: 0 -> 1 (1639643431.89)
goal reach signal edge up
End up at=[ 1.008  1.547  3.117] (0.0312 / 0.0312)
[PLAN] Adjust base once. ((1.0269999999999999, -0.29799999999999999, -0.76100000000000001), (0.0, 0.0, 1.0, 0.0)) / ((1.03, -0.29999999999999999, -0.76000000000000001), (1.0, 0.0))
[PLAN] Qcur: [ 1.027  1.552  3.142]
[PLAN] Qref: [ 1.027  1.552  3.142]
[PLAN] tar: [ 1.027  1.552  3.142]
Send to mobile: [ 1.008  1.547 -1.    -0.012]
             -> [ 1.027  1.552  1.     0.   ]
Distance=0.0311515222968 ([ 0.019  0.005  0.024])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 3.89
goal reach: 1 -> 0 (1639643433.19)
goal reach: 0 -> 1 (1639643433.89)
goal reach signal edge up
End up at=[ 1.026  1.541  3.118] (0.026 / 0.026)
[PLAN] Line idc [0, 4]
[PLAN] Try idc (0, 

Move Tool: -44.8554613894 (1639643439.46)
[KTOOL] send degree: -44.9000708349
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 63)
02:06:254:63:
Move Tool: -44.8047478892 (1639643439.51)
[KTOOL] send degree: -44.8554613894
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 64)
02:06:254:64:
Move Tool: -44.7488728246 (1639643439.55)
[KTOOL] send degree: -44.8047478892
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 64)
02:06:254:64:
Move Tool: -44.6574382567 (1639643439.59)
[KTOOL] send degree: -44.7488728246
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 65)
02:06:254:65:
Move Tool: -44.5927651585 (1639643439.64)
[KTOOL] send degree: -44.6574382567
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 66)
02:06:254:66:
Move Tool: -44.5262037505 (1639643439.68)
[KTOOL] send degree: -44.5927651585
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 67)
02:06:254:67:
Move Tool: -44.4586814236 (1639643439.72)
[KTOOL] send degree: -44.526203750

Move Tool: -40.0256981018 (1639643441.77)
[KTOOL] send degree: -40.1197175534
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 111)
02:06:254:111:
Move Tool: -39.8297377506 (1639643441.83)
[KTOOL] send degree: -40.0256981018
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 112)
02:06:254:112:
Move Tool: -39.7291516299 (1639643441.88)
[KTOOL] send degree: -39.8297377506
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 114)
02:06:254:114:
Move Tool: -39.6276951232 (1639643441.91)
[KTOOL] send degree: -39.7291516299
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 115)
02:06:254:115:
Move Tool: -39.5261762157 (1639643441.95)
[KTOOL] send degree: -39.6276951232
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 116)
02:06:254:116:
Move Tool: -39.4259617599 (1639643442.0)
[KTOOL] send degree: -39.5261762157
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 117)
02:06:254:117:
Move Tool: -39.3269417038 (1639643442.04)
[KTOOL] send degree: -3

Move Tool: -34.2438825628 (1639643444.11)
[KTOOL] send degree: -34.3419092813
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 169)
02:06:254:169:
Move Tool: -34.146477737 (1639643444.15)
[KTOOL] send degree: -34.2438825628
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 170)
02:06:254:170:
Move Tool: -34.0002033389 (1639643444.19)
[KTOOL] send degree: -34.146477737
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 171)
02:06:254:171:
Move Tool: -33.9020534228 (1639643444.23)
[KTOOL] send degree: -34.0002033389
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 172)
02:06:254:172:
Move Tool: -33.8030333667 (1639643444.28)
[KTOOL] send degree: -33.9020534228
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 173)
02:06:254:173:
Move Tool: -33.702818911 (1639643444.32)
[KTOOL] send degree: -33.8030333667
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 174)
02:06:254:174:
Move Tool: -33.6013000034 (1639643444.37)
[KTOOL] send degree: -33.

Move Tool: -29.0165015924 (1639643446.42)
[KTOOL] send degree: -29.0894635137
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 222)
02:06:254:222:
Move Tool: -28.908954278 (1639643446.47)
[KTOOL] send degree: -29.0165015924
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 222)
02:06:254:222:
Move Tool: -28.7703137031 (1639643446.55)
[KTOOL] send degree: -28.908954278
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 223)
02:06:254:223:
Move Tool: -28.7027913761 (1639643446.59)
[KTOOL] send degree: -28.7703137031
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 225)
02:06:254:225:
Move Tool: -28.6362299681 (1639643446.63)
[KTOOL] send degree: -28.7027913761
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 225)
02:06:254:225:
Move Tool: -28.5715568699 (1639643446.67)
[KTOOL] send degree: -28.6362299681
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 226)
02:06:254:226:
Move Tool: -28.5097014626 (1639643446.71)
[KTOOL] send degree: -28

...

Move Tool: -30.0058175326 (1639643610.73)
[KTOOL] send degree: -29.9838832559
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 213)
02:06:254:213:
Move Tool: -30.027856744 (1639643610.78)
[KTOOL] send degree: -30.0058175326
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 212)
02:06:254:212:
Move Tool: -30.049159913 (1639643610.82)
[KTOOL] send degree: -30.027856744
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 212)
02:06:254:212:
Move Tool: -30.0698121611 (1639643610.87)
[KTOOL] send degree: -30.049159913
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 212)
02:06:254:212:
Move Tool: -30.0900333562 (1639643610.91)
[KTOOL] send degree: -30.0698121611
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 212)
02:06:254:212:
Move Tool: -30.1100418473 (1639643610.95)
[KTOOL] send degree: -30.0900333562
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 212)
02:06:254:212:
Move Tool: -30.1302419224 (1639643610.99)
[KTOOL] send degree: -30.1

Move Tool: -31.275281324 (1639643613.07)
[KTOOL] send degree: -31.251970174
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 200)
02:06:254:200:
Move Tool: -31.3102697574 (1639643613.12)
[KTOOL] send degree: -31.275281324
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 200)
02:06:254:200:
Move Tool: -31.3335994448 (1639643613.16)
[KTOOL] send degree: -31.3102697574
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 199)
02:06:254:199:
Move Tool: -31.3569204879 (1639643613.2)
[KTOOL] send degree: -31.3335994448
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 199)
02:06:254:199:
Move Tool: -31.3802343022 (1639643613.24)
[KTOOL] send degree: -31.3569204879
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 199)
02:06:254:199:
Move Tool: -31.4035439482 (1639643613.28)
[KTOOL] send degree: -31.3802343022
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 199)
02:06:254:199:
Move Tool: -31.4268531346 (1639643613.32)
[KTOOL] send degree: -31.4

Move Tool: -32.6173763907 (1639643615.39)
[KTOOL] send degree: -32.5972222033
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 187)
02:06:254:187:
Move Tool: -32.647679996 (1639643615.43)
[KTOOL] send degree: -32.6173763907
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 186)
02:06:254:186:
Move Tool: -32.6683057568 (1639643615.47)
[KTOOL] send degree: -32.647679996
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 186)
02:06:254:186:
Move Tool: -32.6891754798 (1639643615.52)
[KTOOL] send degree: -32.6683057568
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 186)
02:06:254:186:
Move Tool: -32.7101833588 (1639643615.56)
[KTOOL] send degree: -32.6891754798
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 186)
02:06:254:186:
Move Tool: -32.7312071762 (1639643615.6)
[KTOOL] send degree: -32.7101833588
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 185)
02:06:254:185:
Move Tool: -32.7520671393 (1639643615.64)
[KTOOL] send degree: -32.

Move Tool: -33.7966897338 (1639643617.74)
[KTOOL] send degree: -33.7351958895
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 175)
02:06:254:175:
Move Tool: -33.8682303368 (1639643617.78)
[KTOOL] send degree: -33.7966897338
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 175)
02:06:254:175:
Move Tool: -33.9495136619 (1639643617.83)
[KTOOL] send degree: -33.8682303368
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 174)
02:06:254:174:
Move Tool: -34.0402785629 (1639643617.87)
[KTOOL] send degree: -33.9495136619
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 173)
02:06:254:173:
Move Tool: -34.1403061021 (1639643617.91)
[KTOOL] send degree: -34.0402785629
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 172)
02:06:254:172:
Move Tool: -34.3073378936 (1639643617.95)
[KTOOL] send degree: -34.1403061021
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 171)
02:06:254:171:
Move Tool: -34.4287557808 (1639643617.99)
[KTOOL] send degree: -

Move Tool: -40.7914089637 (1639643620.13)
[KTOOL] send degree: -40.6693302046
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 106)
02:06:254:106:
Move Tool: -40.9130163506 (1639643620.17)
[KTOOL] send degree: -40.7914089637
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 105)
02:06:254:105:
Move Tool: -41.0341307248 (1639643620.21)
[KTOOL] send degree: -40.9130163506
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 103)
02:06:254:103:
Move Tool: -41.1552708945 (1639643620.25)
[KTOOL] send degree: -41.0341307248
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 102)
02:06:254:102:
Move Tool: -41.3376223407 (1639643620.3)
[KTOOL] send degree: -41.1552708945
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 101)
02:06:254:101:
Move Tool: -41.4599249387 (1639643620.34)
[KTOOL] send degree: -41.3376223407
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 99)
02:06:254:99:
Move Tool: -41.5830634527 (1639643620.38)
[KTOOL] send degree: -41.

## 4. Sofa cleaning

### 4.1 upside

In [16]:
snode_schedule_list_sofa_all = []

In [17]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

('Height Reference: ', 0.32299999845027921)
[INFO] Depart via: [ 1.4   1.66 -3.13] (32.0) <- [ 1.03  1.8  -3.13] (103.196899414)
Send to mobile: [ 1.027  1.8   -1.     0.003]
             -> [ 1.396  1.662  1.    -0.003]
Distance=0.394124894647 ([ 0.369 -0.138  0.   ])
goal reach: 1 -> 0 (1639643630.59)
goal reach: 0 -> 1 (1639643639.29)
goal reach signal edge up
End up at=[ 1.399  1.622 -3.136] (0.0402 / 0.0402)
[INFO] Approach through: [ 1.36  0.56  3.14] (1.0) -> [ 1.04  0.55  3.14] (101.0)
Send to mobile: [ 1.399  1.622 -1.     0.003]
             -> [ 1.36   0.564  1.     0.   ]
Distance=1.05838848582 ([-0.039 -1.058 -0.005])
goal reach: 1 -> 0 (1639643640.69)
goal reach: 0 -> 1 (1639643682.09)
goal reach signal edge up
End up at=[ 1.383  0.646 -3.122] (6.26 / 6.26)
Send to mobile: [ 1.383  0.646 -1.     0.01 ]
             -> [ 1.042  0.547  1.     0.   ]
Distance=1.25296359134 ([ 0.015 -1.253 -0.007])
goal reach: 1 -> 0 (1639643683.39)
goal reach: 0 -> 1 (1639643691.79)
goal rea

Goal reached
======================= terminated 5: required answers acquired  (0.9/5.0) ===============================
========================== FINISHED (0.9 / 5.0 s) ==============================]
[PLAN] Line idc [2, 5]
[PLAN] Try idc (2, 5)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 2: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 9: required answers acquired  (0.9/5.0) ===============================
======================= terminated 4: required answers acquired  (0.9/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.9/5.0) ===============================
Goal reached
======================= terminated 0: required answers acquired  (0.9/5.0) ===============================
Goal reached
======================= terminated 6: required answers acquired  (1.

Move Tool: -37.9201984522 (1639643703.36)
[KTOOL] send degree: -38.1760375662
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
02:06:254:131:
Move Tool: -37.6665862602 (1639643703.4)
[KTOOL] send degree: -37.9201984522
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 133)
02:06:254:133:
Move Tool: -37.414235031 (1639643703.44)
[KTOOL] send degree: -37.6665862602
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 136)
02:06:254:136:
Move Tool: -37.1619202309 (1639643703.48)
[KTOOL] send degree: -37.414235031
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 138)
02:06:254:138:
Move Tool: -36.9074276387 (1639643703.53)
[KTOOL] send degree: -37.1619202309
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 141)
02:06:254:141:
Move Tool: -36.5220393544 (1639643703.58)
[KTOOL] send degree: -36.9074276387
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 143)
02:06:254:143:
Move Tool: -36.2634284118 (1639643703.62)
[KTOOL] send degree: -36.

Move Tool: -22.2501418558 (1639643705.68)
[KTOOL] send degree: -22.5311755243
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 31)
02:06:255:31:
Move Tool: -21.9692435339 (1639643705.72)
[KTOOL] send degree: -22.2501418558
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 34)
02:06:255:34:
Move Tool: -21.6883928473 (1639643705.77)
[KTOOL] send degree: -21.9692435339
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 37)
02:06:255:37:
Move Tool: -21.407429076 (1639643705.81)
[KTOOL] send degree: -21.6883928473
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 40)
02:06:255:40:
Move Tool: -21.1263205933 (1639643705.85)
[KTOOL] send degree: -21.407429076
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 42)
02:06:255:42:
Move Tool: -20.7043383856 (1639643705.89)
[KTOOL] send degree: -21.1263205933
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 45)
02:06:255:45:
Move Tool: -20.2819229824 (1639643705.95)
[KTOOL] send degree: -20.7043383856


Move Tool: -6.13148344625 (1639643708.06)
[KTOOL] send degree: -6.38383467541
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 193)
02:06:255:193:
Move Tool: -5.8778712542 (1639643708.11)
[KTOOL] send degree: -6.13148344625
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 195)
02:06:255:195:
Move Tool: -5.62203214022 (1639643708.15)
[KTOOL] send degree: -5.8778712542
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 198)
02:06:255:198:
Move Tool: -5.36298640289 (1639643708.19)
[KTOOL] send degree: -5.62203214022
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 200)
02:06:255:200:
Move Tool: -5.0997417251 (1639643708.23)
[KTOOL] send degree: -5.36298640289
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 203)
02:06:255:203:
Move Tool: -4.83205750514 (1639643708.27)
[KTOOL] send degree: -5.0997417251
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 206)
02:06:255:206:
Move Tool: -4.56458048119 (1639643708.31)
[KTOOL] send degree: -4.83

...

Move Tool: -8.55403311619 (1639643762.72)
[KTOOL] send degree: -8.65153692332
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 170)
02:06:255:170:
Move Tool: -8.45216547896 (1639643762.76)
[KTOOL] send degree: -8.55403311619
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 171)
02:06:255:171:
Move Tool: -8.34719642463 (1639643762.8)
[KTOOL] send degree: -8.45216547896
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 172)
02:06:255:172:
Move Tool: -8.24086670912 (1639643762.85)
[KTOOL] send degree: -8.34719642463
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 173)
02:06:255:173:
Move Tool: -8.13296415389 (1639643762.89)
[KTOOL] send degree: -8.24086670912
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 174)
02:06:255:174:
Move Tool: -8.02168213729 (1639643762.93)
[KTOOL] send degree: -8.13296415389
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 175)
02:06:255:175:
Move Tool: -7.90518646956 (1639643762.97)
[KTOOL] send degree: -8

Move Tool: 0.947191936401 (1639643765.08)
[KTOOL] send degree: 0.779866202357
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 7)
02:06:00:07:
Move Tool: 1.11066256146 (1639643765.12)
[KTOOL] send degree: 0.947191936401
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 9)
02:06:00:09:
Move Tool: 1.340337349 (1639643765.19)
[KTOOL] send degree: 1.11066256146
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 11)
02:06:00:11:
Move Tool: 1.47755313373 (1639643765.23)
[KTOOL] send degree: 1.340337349
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 13)
02:06:00:13:
Move Tool: 1.65971864043 (1639643765.28)
[KTOOL] send degree: 1.47755313373
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 14)
02:06:00:14:
Move Tool: 1.77681643623 (1639643765.32)
[KTOOL] send degree: 1.65971864043
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 16)
02:06:00:16:
Move Tool: 1.88859245079 (1639643765.36)
[KTOOL] send degree: 1.77681643623
[KTOOL] send (<MOTOR_CMD.OP: 2>, 

Move Tool: -10.8384630986 (1639643767.6)
[KTOOL] send degree: -10.3625726405
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 153)
02:06:255:153:
Move Tool: -11.1640219253 (1639643767.64)
[KTOOL] send degree: -10.8384630986
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 148)
02:06:255:148:
Move Tool: -11.4936150445 (1639643767.68)
[KTOOL] send degree: -11.1640219253
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 145)
02:06:255:145:
Move Tool: -11.8250950203 (1639643767.72)
[KTOOL] send degree: -11.4936150445
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 142)
02:06:255:142:
Move Tool: -12.1541389499 (1639643767.77)
[KTOOL] send degree: -11.8250950203
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 138)
02:06:255:138:
Move Tool: -12.4785816425 (1639643767.81)
[KTOOL] send degree: -12.1541389499
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 135)
02:06:255:135:
Move Tool: -12.7995423841 (1639643767.85)
[KTOOL] send degree: -1

Move Tool: -29.6804598897 (1639643769.94)
[KTOOL] send degree: -29.3585965183
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 219)
02:06:254:219:
Move Tool: -30.0022679259 (1639643769.98)
[KTOOL] send degree: -29.6804598897
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 216)
02:06:254:216:
Move Tool: -30.3239720323 (1639643770.02)
[KTOOL] send degree: -30.0022679259
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 212)
02:06:254:212:
Move Tool: -30.6455217673 (1639643770.06)
[KTOOL] send degree: -30.3239720323
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 209)
02:06:254:209:
Move Tool: -30.9668646288 (1639643770.1)
[KTOOL] send degree: -30.6455217673
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 206)
02:06:254:206:
Move Tool: -31.287982277 (1639643770.14)
[KTOOL] send degree: -30.9668646288
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 203)
02:06:254:203:
Move Tool: -31.769689586 (1639643770.2)
[KTOOL] send degree: -31.2

Move Tool: -45.0 (1639643772.31)
[KTOOL] send degree: -45.0163583687
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
02:06:254:62:
Connect: Server IP (192.168.0.3)



In [18]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_back_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [19]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_right_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [20]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_left_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [21]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_back_front, tip_dir="up", sweep_dir="Z", tool_dir=1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

Send to mobile: [ 1.051  0.561 -1.     0.014]
             -> [ 1.284  0.31   1.     0.   ]
Distance=0.344192645374 ([ 0.233 -0.251 -0.028])
goal reach: 1 -> 0 (1639643774.39)
goal reach: 0 -> 1 (1639643780.19)
goal reach signal edge up
End up at=[ 1.303  0.292 -3.109] (6.25 / 6.25)
[PLAN] Adjust base once. ((0.79500000000000004, -0.39000000000000001, 1.2290000000000001), (-0.70699999999999996, 0.0, 0.70699999999999996, 0.0)) / ((0.79000000000000004, -0.39000000000000001, 1.23), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 1.284  0.31   3.142]
[PLAN] Qref: [ 1.284  0.31   3.142]
[PLAN] tar: [ 1.284  0.31   3.142]
Send to mobile: [ 1.302  0.291 -1.     0.016]
             -> [ 1.284  0.31   1.     0.   ]
Distance=0.0414539205872 ([-0.018  0.019 -0.032])
goal reach: 1 -> 0 (1639643781.59)
goal reach: 0 -> 1 (1639643782.49)
goal reach signal edge up
End up at=[ 1.278  0.318 -3.103] (6.24 / 6.24)
[PLAN] Line idc [0, 3]
[PLAN] Try idc (0, 3)
Use 10/12 agents
======================= terminated

======================= terminated 6: Stop called from other agent  (0.6/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.6/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (0.7/5.0) ===============================
Goal reached
======================= terminated 7: required answers acquired  (0.7/5.0) ===============================
Goal reached
Goal reached
======================= terminated 1: required answers acquired  (0.7/5.0) ===============================
======================= terminated 0: required answers acquired  (0.7/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.7/5.0) ===============================
Goal reached
======================= terminated 9: required answers acquired  (0.7/5.0) ===============================
========================== FINISHED (0.7 / 5.0 s) ==========

======================= terminated 6: required answers acquired  (1.2/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (1.2/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (1.2/5.0) ===============================
======================= terminated 8: Stop called from other agent  (1.2/5.0) ===============================
========================== FINISHED (1.2 / 5.0 s) ==============================]
Goal reached
Move Tool: -44.9918700036 (1639643793.99)
[KTOOL] send degree: -45.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
02:06:254:62:
Move Tool: -44.9291757715 (1639643794.04)
[KTOOL] send degree: -44.9918700036
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 63)
02:06:254:63:
Move Tool: -44.8097978236 (1639643794.08)
[KTOOL] send degree: -44.9291757715
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 63)
02:06:254

Move Tool: -24.5641848475 (1639643795.95)
[KTOOL] send degree: -25.2940650195
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 4)
02:06:255:04:
Move Tool: -24.0770121769 (1639643796.0)
[KTOOL] send degree: -24.5641848475
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 11)
02:06:255:11:
Move Tool: -23.5888750708 (1639643796.04)
[KTOOL] send degree: -24.0770121769
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 16)
02:06:255:16:
Move Tool: -23.0995104235 (1639643796.08)
[KTOOL] send degree: -23.5888750708
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 21)
02:06:255:21:
Move Tool: -22.6100403919 (1639643796.12)
[KTOOL] send degree: -23.0995104235
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 26)
02:06:255:26:
Move Tool: -22.1212711209 (1639643796.16)
[KTOOL] send degree: -22.6100403919
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 30)
02:06:255:30:
Move Tool: -21.63400979 (1639643796.21)
[KTOOL] send degree: -22.1212711209
[K

Move Tool: 6.43720062104 (1639643798.3)
[KTOOL] send degree: 5.89129813783
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 58)
02:06:00:58:
Move Tool: 6.98191110825 (1639643798.34)
[KTOOL] send degree: 6.43720062104
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 64)
02:06:00:64:
Move Tool: 7.52406449065 (1639643798.38)
[KTOOL] send degree: 6.98191110825
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 69)
02:06:00:69:
Move Tool: 8.06222953904 (1639643798.42)
[KTOOL] send degree: 7.52406449065
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 75)
02:06:00:75:
Move Tool: 8.5957525382 (1639643798.47)
[KTOOL] send degree: 8.06222953904
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 80)
02:06:00:80:
Move Tool: 9.12974144258 (1639643798.51)
[KTOOL] send degree: 8.5957525382
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 85)
02:06:00:85:
Move Tool: 9.667662918 (1639643798.55)
[KTOOL] send degree: 9.12974144258
[KTOOL] send (<MOTOR_CMD.OP: 2>, <O

...

Move Tool: 44.7790276092 (1639643835.4)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643835.45)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643835.49)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643835.54)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643835.58)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643835.62)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643835.66)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTO

Move Tool: 44.7790276092 (1639643837.76)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643837.8)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643837.84)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643837.88)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643837.92)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643837.96)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643838.01)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTO

Move Tool: 44.7790276092 (1639643840.13)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643840.17)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643840.21)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643840.26)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643840.3)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643840.34)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643840.39)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTO

Move Tool: 44.7790276092 (1639643842.68)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643842.72)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643842.78)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643842.82)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643842.87)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643842.92)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 1, 191)
02:06:01:191:
Move Tool: 44.7790276092 (1639643842.96)
[KTOOL] send degree: 44.7790276092
[KTOOL] send (<MOT

KeyboardInterrupt: 

In [ ]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_right_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [ ]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [ ]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_left_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [ ]:
save_schedules(snode_schedule_list_table_all, snode_schedule_list_sofa_all)

In [ ]:
snode_schedule_list_table_all, snode_schedule_list_sofa_all = load_schedules()

In [ ]:
for snode_schedule_list in snode_schedule_list_table_all:
    play_cleaning_schedule(ppline, table_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

In [ ]:
for snode_schedule_list in snode_schedule_list_sofa_all:
    play_cleaning_schedule(ppline, sofa_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

### Finish

In [28]:
if CONNECT_TASK_PLANNER:
    servicer.mark_task_finished()

* 맵 폴 추가 v
* 플래닝 마치고 동작 시 빨간 하이라이트 없애기 v
* 스케줄 세이브 / 로드 구현하기 v
* NoSDK 7DoF 구현하기 v

* no-sdk 버전 mode switcher에 닦기 속도 조절 구현하기
* sdk 버전 제어기 개선 - 아래쪽, 먼 곳 닦을 때 토크 과다?